# PyWry AG Grid Demo

This notebook demonstrates PyWry's built-in AG Grid integration for displaying DataFrames with:

- **Automatic type detection** - datetime, numbers, booleans auto-configure sorting/filtering
- **MultiIndex support** - column groups and row spanning
- **Custom columns** - computed values with `value_getter`
- **Row selection** - click, ctrl+click, checkboxes
- **Theming** - dark/light modes with multiple AG Grid themes

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from pywry import show_dataframe
from pywry.grid import ColDef, RowSelection

## 1. Basic DataFrame

Just pass a DataFrame - everything works out of the box!

In [ ]:
df_simple = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'age': [30, 25, 35, 28, 42],
    'city': ['NYC', 'LA', 'Chicago', 'Boston', 'Seattle'],
    'active': [True, False, True, True, False],
    'salary': [75000, 65000, 85000, 72000, 95000],
})

show_dataframe(df_simple);

## 2. DateTime Handling

Datetime columns are automatically serialized to ISO format and detected by AG Grid for proper sorting and filtering.

In [ ]:
df_dates = pd.DataFrame({
    'event': ['Launch', 'Update', 'Release', 'Patch', 'Hotfix'],
    'timestamp': pd.to_datetime([
        '2025-01-15 09:30:00', '2025-02-20 14:45:30', 
        '2025-03-10 08:00:00', '2025-04-05 16:20:15',
        '2025-05-01 11:11:11'
    ]),
    'date_only': pd.to_datetime([
        '2025-01-15', '2025-02-20', '2025-03-10', '2025-04-05', '2025-05-01'
    ]).date,
    'value': [100, 150, 200, 175, 225],
})

# Add a timedelta column
df_dates['duration'] = df_dates['timestamp'] - df_dates['timestamp'].iloc[0]

show_dataframe(df_dates);

## 3. MultiIndex Columns (Column Groups)

MultiIndex columns automatically become AG Grid column groups.

In [ ]:
# Create MultiIndex columns
col_index = pd.MultiIndex.from_tuples([
    ('Revenue', 'US'), ('Revenue', 'EU'), ('Revenue', 'APAC'),
    ('Cost', 'US'), ('Cost', 'EU'), ('Cost', 'APAC')
])

df_multicol = pd.DataFrame(
    np.random.randint(50, 200, size=(4, 6)),
    columns=col_index,
    index=['Q1', 'Q2', 'Q3', 'Q4']
)

show_dataframe(df_multicol);

## 4. MultiIndex Rows with Row Spanning

MultiIndex rows become pinned columns on the left. When `enable_cell_span=True` (auto-detected), repeated values are merged.

In [ ]:
# Create MultiIndex rows
row_index = pd.MultiIndex.from_tuples([
    ('2024', 'Q1'), ('2024', 'Q2'), ('2024', 'Q3'), ('2024', 'Q4'),
    ('2025', 'Q1'), ('2025', 'Q2'), ('2025', 'Q3'), ('2025', 'Q4'),
], names=['Year', 'Quarter'])

df_multirow = pd.DataFrame({
    'Revenue': np.random.randint(100, 200, 8),
    'Cost': np.random.randint(50, 100, 8),
    'Profit': np.random.randint(20, 80, 8),
}, index=row_index)

# Row spanning is auto-enabled for MultiIndex rows
show_dataframe(df_multirow);

## 5. Both MultiIndex (Columns AND Rows)

Combine column groups with row spanning for complex hierarchical data.

In [ ]:
# Both MultiIndex columns AND rows
row_index = pd.MultiIndex.from_tuples([
    ('2024', 'Q1'), ('2024', 'Q2'), ('2025', 'Q1'), ('2025', 'Q2')
], names=['Year', 'Quarter'])

col_index = pd.MultiIndex.from_tuples([
    ('Revenue', 'US'), ('Revenue', 'EU'),
    ('Cost', 'US'), ('Cost', 'EU')
])

df_both = pd.DataFrame(
    [[100, 80, 50, 40],
     [110, 85, 55, 42],
     [120, 90, 60, 45],
     [130, 95, 65, 48]],
    index=row_index, columns=col_index
)

show_dataframe(df_both);

## 6. Custom Column Definitions

Use `ColDef` for fine-grained control over columns - widths, pinning, formatting, etc.

In [ ]:
df_products = pd.DataFrame({
    'sku': ['A001', 'B002', 'C003', 'D004', 'E005'],
    'name': ['Widget', 'Gadget', 'Gizmo', 'Thingamajig', 'Doohickey'],
    'price': [19.99, 29.99, 14.99, 49.99, 9.99],
    'quantity': [100, 50, 200, 25, 500],
    'category': ['Electronics', 'Electronics', 'Home', 'Industrial', 'Home'],
})

custom_cols = [
    ColDef(field='sku', header_name='SKU', pinned='left', width=80),
    ColDef(field='name', header_name='Product Name', width=150),
    ColDef(field='price', header_name='Price ($)', cell_data_type='number',
           value_formatter="'$' + value.toFixed(2)"),
    ColDef(field='quantity', header_name='Qty', cell_data_type='number', width=80),
    ColDef(field='category', header_name='Category'),
]

show_dataframe(df_products, column_defs=custom_cols);

## 7. Computed Columns with `value_getter`

Use JavaScript expressions to create calculated columns that update dynamically.

In [ ]:
df_sales = pd.DataFrame({
    'product': ['Laptop', 'Phone', 'Tablet', 'Watch', 'Headphones'],
    'units': [150, 500, 300, 200, 400],
    'unit_price': [999, 699, 449, 299, 149],
    'discount_pct': [0.10, 0.15, 0.05, 0.20, 0.10],
})

computed_cols = [
    ColDef(field='product', header_name='Product', pinned='left'),
    ColDef(field='units', header_name='Units Sold', cell_data_type='number'),
    ColDef(field='unit_price', header_name='Unit Price',
           value_formatter="'$' + value.toLocaleString()"),
    ColDef(field='discount_pct', header_name='Discount',
           value_formatter="(value * 100).toFixed(0) + '%'"),
    # Computed: Gross Revenue
    ColDef(
        header_name='Gross Revenue',
        value_getter="data.units * data.unit_price",
        value_formatter="'$' + value.toLocaleString()",
        cell_data_type='number'
    ),
    # Computed: Net Revenue (after discount)
    ColDef(
        header_name='Net Revenue',
        value_getter="data.units * data.unit_price * (1 - data.discount_pct)",
        value_formatter="'$' + value.toLocaleString(undefined, {minimumFractionDigits: 0, maximumFractionDigits: 0})",
        cell_data_type='number'
    ),
]

show_dataframe(df_sales, column_defs=computed_cols);

## 8. Handling NaN/NaT Values

Missing values (NaN, NaT, None) are properly handled and shown as blank cells.

In [ ]:
df_missing = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'Diana'],
    'score': [95.5, np.nan, 87.0, None],
    'last_login': ['2025-01-15', '2025-01-10', pd.NaT, '2025-01-12'],
    'status': ['active', None, 'active', 'inactive'],
})
show_dataframe(df_missing);

## 9. Row Selection Options

Configure row selection behavior - single row, multi-row, with/without checkboxes.

In [ ]:
# Single row selection (no checkboxes)
single_select = RowSelection(
    mode='singleRow',
    checkbox_selection=False,
    enable_click_selection=True,
)

show_dataframe(df_simple, row_selection=single_select);

In [ ]:
# Multi-row selection with checkboxes (default)
show_dataframe(df_simple, row_selection=True);

In [ ]:
# Disable row selection entirely (default)
show_dataframe(df_simple, row_selection=False);

## 10. Pagination

Enable pagination for large datasets.

In [ ]:
# Generate a larger dataset - pagination auto-enables for >10 rows
df_large = pd.DataFrame({
    'id': range(1, 205),
    'value': np.random.randn(204).round(3),
    'category': np.random.choice(['A', 'B', 'C', 'D'], 204),
    'timestamp': pd.date_range('2025-01-01', periods=204, freq='h'),
})

show_dataframe(df_large);

## 11. Theming

Choose between dark/light mode and different AG Grid themes.

In [ ]:
# Light theme with Quartz AG Grid theme
show_dataframe(df_simple, theme='light', aggrid_theme='quartz');

In [ ]:
# Dark theme with Balham AG Grid theme
show_dataframe(df_simple, theme='dark', aggrid_theme='balham');

## 12. Callback System - Interactive Grid Updates

Use callbacks to respond to user interactions:
- `grid:cell-click` - cell clicked (includes `rowIndex`, `colId`, `value`, `data`)
- `grid:row-selected` - row selection changed (includes `rows` array)
- `grid:state-response` - response to `request_grid_state()` call

Update methods:
- `widget.update_cell(row_index, col_id, value)` - efficient single-cell update
- `widget.update_grid(data, columns, restore_state)` - full grid update with state restoration
- `widget.request_grid_state(context)` - request current column/filter/sort state

**Toolbar Items** - Use Pydantic models for type-safe toolbar configuration:
```python
from pywry import Toolbar, Button, Select, MultiSelect, TextInput, NumberInput, DateInput, RangeInput, Option

toolbar = Toolbar(
    position="top",
    items=[
        Button(label="Export", event="toolbar:export", data={"format": "csv"}),
        Select(
            label="View:",
            event="view:change",
            options=[Option(label="Table", value="table"), Option(label="Chart", value="chart")],
            selected="table",
        ),
        MultiSelect(
            label="Columns:",
            event="columns:filter",
            options=[Option(label="Name"), Option(label="Age")],
            selected=["Name"],
        ),
        TextInput(label="Search:", event="search:query", placeholder="...", debounce=300),
        NumberInput(label="Limit:", event="filter:limit", value=10, min_val=1, max_val=100),
        DateInput(label="Date:", event="filter:date", value="2025-01-01"),
        RangeInput(label="Zoom:", event="zoom:level", value=50, min_val=0, max_val=100, step=5),
    ],
)
```

Each item has:
- `component_id`: Auto-generated unique ID for state tracking
- `event`: Validated format `namespace:event-name` (e.g., `"view:change"`)
- `description`: Optional tooltip text shown on hover
- `disabled`: Whether the item is disabled

In [ ]:
# Example 1: Single-cell update using `update_cell()`
# Click the 'clicks' column to increment the counter

df_clicks = pd.DataFrame({
    "ID": [1, 2, 3, 4],
    "item": ["Apples", "Bananas", "Oranges", "Grapes"],
    "clicks": [0, 0, 0, 0],
    "price": [1.50, 0.75, 2.00, 3.25],
})

def on_cell_click(data, event_type, widget_id):
    """Handle cell click - get value from event, increment, update cell."""
    col_id = data.get("colId")
    row_index = data.get("rowIndex")
    row_data = data.get("data")  # Full row data from the grid
    
    if col_id == "clicks" and row_index is not None and row_data:
        # Get current value from the event, increment it
        current_value = row_data.get("clicks", 0)
        new_value = current_value + 1
        # Update just this cell
        cell_widget.update_cell(row_index, col_id, new_value)

cell_widget = show_dataframe(
    df_clicks,
    callbacks={"grid:cell-click": on_cell_click},
    row_selection=False,
)

In [ ]:
# Example 2: Multiple views with state preservation
# Switch between different views while preserving column order, widths, filters per view

from pywry import Toolbar, Select, Option
from pywry.grid import build_grid_config, to_js_grid_config

# Sample transaction data
transactions = pd.DataFrame({
    "date": pd.date_range("2025-01-01", periods=20, freq="D").strftime("%Y-%m-%d"),
    "product": ["Laptop", "Phone", "Tablet", "Watch"] * 5,
    "region": ["North", "South", "East", "West"] * 5,
    "units": np.random.randint(10, 100, 20),
    "price": [999, 699, 449, 299] * 5,
})
transactions["revenue"] = transactions["units"] * transactions["price"]

# Pre-compute views
views = {
    "transactions": transactions.copy(),
    "product_pivot": transactions.pivot_table(
        index="product", columns="region", values="revenue", aggfunc="sum"
    ).reset_index(),
}

# State management
view_states = {}  # Stores grid state per view
current_view = {"name": "transactions", "pending_switch": None}

def on_view_change(data, event_type, widget_id):
    """Handle view switch - request current state first, then switch."""
    new_view = data.get("value")  # Select dropdown sends {value: ...}
    if new_view == current_view["name"]:
        return  # Already on this view
    
    # Store pending switch and request current grid state
    current_view["pending_switch"] = new_view
    view_widget.request_grid_state({"target_view": new_view})

def on_state_response(data, event_type, widget_id):
    """Handle grid state response - save it and complete the view switch."""
    target_view = data.get("context", {}).get("target_view")
    if not target_view or not current_view["pending_switch"]:
        return
    
    # Save current view's state
    view_states[current_view["name"]] = {
        "columnState": data.get("columnState"),
        "filterModel": data.get("filterModel"),
        "sortModel": data.get("sortModel"),
    }
    
    # Switch to new view
    current_view["name"] = target_view
    current_view["pending_switch"] = None
    
    # Build new view config
    config = build_grid_config(views[target_view])
    js_config = to_js_grid_config(config)
    
    # Update grid with new data and restore saved state (if any)
    view_widget.update_grid(
        data=js_config["rowData"],
        columns=js_config["columnDefs"],
        restore_state=view_states.get(target_view),
    )

# Toolbar using Pydantic models - type-safe with auto-generated component IDs
toolbar = Toolbar(
    position="left",
    items=[
        Select(
            label="",
            event="view:change",
            description="Switch between different data views",
            options=[
                Option(label="Transactions", value="transactions"),
                Option(label="Product Pivot", value="product_pivot"),
            ],
            selected="transactions",
        )
    ],
)

view_widget = show_dataframe(
    views["transactions"],
    toolbars=[toolbar],
    callbacks={
        "view:change": on_view_change,
        "grid:state-response": on_state_response,
    },
);